In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

In [ ]:

# >>> THAY ĐỔI Ở ĐÂY (1): Điền đúng tên file dữ liệu của bài thi <<<
file_path = 'online_shoppers_purchasing_intention.csv'

# >>> THAY ĐỔI Ở ĐÂY (2): Điền đúng tên cột kết quả (cột y) <<<
target_column = 'Revenue'

# --- Phần còn lại của bước này cứ để nó tự chạy ---
print("--- Bắt đầu đọc và phân tích dữ liệu ---")
df = pd.read_csv(file_path)

In [ ]:

# Chuyển đổi cột target sang dạng số (0 và 1) để máy học hiểu
# Giả sử cột target có 2 giá trị dạng chữ hoặc True/False
if df[target_column].dtype != 'int64' and df[target_column].dtype != 'float64':
    # Lấy ra 2 giá trị duy nhất và map cái thứ 2 thành 1, cái thứ 1 thành 0
    unique_values = df[target_column].unique()
    mapping = {unique_values[0]: 0, unique_values[1]: 1}
    df[target_column] = df[target_column].map(mapping)
    print(f"Đã chuyển đổi cột '{target_column}' thành số với mapping: {mapping}")


In [ ]:
# **KIỂM TRA QUAN TRỌNG NHẤT: Dữ liệu có mất cân bằng không?**
plt.figure(figsize=(8, 5))
sns.countplot(x=target_column, data=df)
plt.title(f'Phân phối của cột kết quả "{target_column}" (0 vs 1)')
plt.show()
print("Nếu 2 cột trên chênh lệch nhiều, BẮT BUỘC phải xử lý mất cân bằng (code bên dưới đã xử lý).")

In [ ]:

# ==============================================================================
# === BƯỚC 2: CHUẨN BỊ DỮ LIỆU (KHÔNG CẦN SỬA GÌ CẢ) ===
# ==============================================================================
print("\n--- Bắt đầu quá trình tiền xử lý dữ liệu tự động ---")
# Tách X (features) và y (target)
X = df.drop(target_column, axis=1)
y = df[target_column]

In [ ]:

# Tự động xác định cột số và cột chữ
numerical_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = X.select_dtypes(include='object').columns.tolist()
print(f"Đã tìm thấy {len(numerical_features)} cột số và {len(categorical_features)} cột chữ.")

In [ ]:

# Tạo cỗ máy tiền xử lý tự động
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

In [ ]:

# Chia dữ liệu (luôn dùng stratify=y để giữ nguyên tỉ lệ mất cân bằng)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
print("Đã chia dữ liệu thành tập train và test.")


In [ ]:
# ==============================================================================
# === BƯỚC 3, 4, 5: PIPELINE "TẤT CẢ TRONG MỘT" (KHÔNG CẦN SỬA GÌ CẢ) ===
# ==============================================================================
print("\n--- Bắt đầu xây dựng và tinh chỉnh mô hình (quá trình này có thể mất vài phút) ---")
# Tạo pipeline "thần thánh": Tiền xử lý -> SMOTE -> Huấn luyện
pipeline = ImbPipeline(steps=[
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=42)),
    ('classifier', RandomForestClassifier(random_state=42, class_weight='balanced'))
])


In [ ]:
# Định nghĩa các "vũ khí" để nâng cấp mô hình
# Đây là các thiết lập khác nhau để GridSearchCV tự động thử và chọn ra cái tốt nhất
param_grid = {
    'classifier__n_estimators': [100, 150],
    'classifier__max_depth': [10, 20],
    'classifier__min_samples_leaf': [2, 4]
}


In [ ]:
# Khởi tạo cỗ máy tìm kiếm GridSearchCV
# Nó sẽ tự động tìm bộ tham số trong param_grid để tối đa hóa F1-Score
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='f1', # **CHỈ THỊ CỐT LÕI: Tối ưu hóa theo F1-Score!**
    cv=3,         # 3 lần là đủ nhanh cho bài kiểm tra
    n_jobs=-1,    # Dùng tất cả CPU cho nhanh
    verbose=1
)


In [ ]:
# Bắt đầu chạy!
grid_search.fit(X_train, y_train)


In [ ]:
print(f"\nQuá trình tinh chỉnh hoàn tất!")
print(f"Tham số tốt nhất tìm được: {grid_search.best_params_}")
print(f"F1-score tốt nhất trên tập validation (cross-validation): {grid_search.best_score_:.4f}")

In [ ]:
# ==============================================================================
# === BƯỚC 6: ĐÁNH GIÁ CUỐI CÙNG (CHỈ SỬA 1 DÒNG NẾU CẦN) ===
# ==============================================================================

# >>> THAY ĐỔI Ở ĐÂY (3): Sửa mục tiêu F1-Score nếu đề bài yêu cầu khác <<<
target_f1_score = 0.65

# --- Phần còn lại cứ để nó tự chạy ---
print("\n--- Đánh giá mô hình cuối cùng trên tập test ---")
best_model = grid_search.best_estimator_
y_pred_final = best_model.predict(X_test)

print("\nBáo cáo kết quả chi tiết:")
print(classification_report(y_test, y_pred_final))

print("Ma trận nhầm lẫn (Confusion Matrix):")
cm = confusion_matrix(y_test, y_pred_final)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

final_f1_score = f1_score(y_test, y_pred_final)
print(f"\n>>> F1-SCORE CUỐI CÙNG TRÊN TẬP TEST (LỚP 1): {final_f1_score:.4f} <<<")

if final_f1_score >= target_f1_score:
    print(f"\nCHÚC MỪNG! BẠN ĐÃ ĐẠT MỤC TIÊU (>= {target_f1_score}). Nộp bài thôi!")
else:
    print(f"\nTIẾC QUÁ! Kết quả chưa đạt mục tiêu {target_f1_score}. Nhưng bạn đã làm hết các bước cần thiết!")